In [10]:
#TAOP89 Marcus Döberl (mardo435) York F.v Wangenheim (yorfr185)
import numpy as np
import time
import sys
import copy

e=1

prob=" ".join(sys.argv[1:]).split('.')[0]
fil=prob+'.npz'

npzfile = np.load(fil)
npzfile.files
m=npzfile['m'] #antal möjliga locations för en fabrik
n=npzfile['n'] #antalet kunder 
s=npzfile['s'] #kapaciteten hos en fabrik på plats i    
d=npzfile['d'] #efterfrågan hos en kund j
f=npzfile['f'] #fasta kostnade för en anläggning på plats i
c=npzfile['c'] #transportkostnaden per enhet till kund j från anläggning i
#print 'm:',m,' n:',n
#print 's:',s
#print 'd:',d
#print 'f:',f
#print 'c:',c

t1=time.time()
x=np.zeros((m,n),dtype=np.int)
y=np.zeros((m),dtype=np.int)

ss=copy.deepcopy(s)
dd=copy.deepcopy(d)

while sum(dd)>0:
    # find facility, find customer, send, at min cost
    # set x and y
    # deduct from ss and dd, 
    # --------



elapsed = time.time() - t1
print 'Tid: '+str('%.4f' % elapsed)

cost=sum(sum(np.multiply(c,x))) + e*np.dot(f,y)
print 'Problem:',prob,' Totalkostnad: '+str(cost)
print 'y:',y
print 'Antal byggda fabriker:',sum(y),'(av',m,')'

enter file name: Reading data from file 'lab1-mus.mat'
Datafile: 'lab1-mus.mat'


error: Error -3 while decompressing data: incorrect header check